# 14.1 Segmentació d'instàncies

Com ja havíem introduït en capítols anteriors, la segmentació d'imatges crea una imatge píxel a píxel dels objectes detectats i té dos nivells d'actuació: la segmentació semàntica i la segmentació d'instàncies. En la segmentació semàntica s'analitza la imatge a nivell de píxels, mentre que en la segmentació d'instàncies primer es fa una detecció d'objectes i posteriorment s'etiqueten els píxels dins l'objecte detectat.

<table width=75%>
    <tr>
        <td align=center style="border:1px solid black">
            <img src="Imatges/cats.jpeg"><br>
            Original
        </td>
        <td align=center style="border:1px solid black">
            <img src="Imatges/cats_objdetection.png"><br>
            Detecció d'objectes
        </td>
    </tr>
     <tr >
        <td align=center style="border:1px solid black; background:white">
            <img src="Imatges/cats_instance_segmentation.jpeg"><br>
            Segmentació
        </td>
        <td align=center style="border:1px solid black; background:white">
            <img src="Imatges/cats_img_segmentation.png"><br>
            Segmentació d'instàncies
        </td>
    </tr>
    <tr>
        <td colspan=2 align=center style="border:1px solid black">
            <img src="Imatges/cats_panoptic.png"><br>
            Segmentació Panòptica
        </td>
    </tr>
   <tr>
        <td colspan=2 align=center style="border:1px solid black">
            <i>Font: <a href="https://medium.com/@danielmechea/what-is-panoptic-segmentation-and-why-you-should-care-7f6c953d2a6a">What is Panoptic Segmentation and why you should care</a>, <br>de Daniel Mechea en Medium</i>
        </td>
    </tr>
</table>

       
Abans de la segmentació d'instàncies els sistemes només detectaven objectes en imatges i els marcaven amb una capsa de mides i forma fixes, però això no era suficient per prendre decisions en molts sistemes com els de conducció automàtica o en les imatges mèdiques (entre d'altres), necessitaven saber-ne la forma exacta. L'aparició de la segmentació d'instàncies va tenir una gran repercussió en aquesta camps: saber la forma precisa d'un cotxe detectat podia ajudar al sistema de conducció automàtica a esquivar-lo amb precisió; saber la forma precisa de les cèl·lules podia ajudar a detectar les cèl·lules cancerígens de manera més eficaç, etc.


Són molts els camps que treuen profit de la segmentació d'instàncies, entre ells podem anomenar:
- Sistemes de control de trànsit
- Conducció automàtica
- Detecció d'objectes per satèl·lit
- Biomedicina i imatges mèdiques 

## Mètriques per Segmentació d'Imatges
Tot algoritme necessita algun tipus de mètrica d'error per comprovar com de bé està assolint el seu objectiu. En el cas de la segmentació d'imatges hi ha diverses mètriques que es poden aplicar, però sobretot podríem destacar-ne dues: 

### Pixel Accuracy
El Pixel Accuracy mesura el percentatge de píxels que han estat segmentats correctament en una imatge.

Tot i que sembla molt lògic d'entendre i aplicar, té un gran desavantatge quan les classes a detectar no estan equilibrades en una imatge. Per exemple, si mirem la imatge de sota, a l'esquerra tenim una imatge satèl·lit amb un port amb vaixells que volem detectar de manera automàtica amb segmentació d'imatges. A la dreta veiem el que s'anomena "ground truth", el que detectaríem manualment. 

<table width=75%>
    <tr>
        <td>
            <img src="Imatges/vaixell-satellit.png">
        </td>
    </tr>
    <tr>
        <td>
            Font: Article <a href="https://towardsdatascience.com/metrics-to-evaluate-your-semantic-segmentation-model-6bcb99639aa2">Image Segmentation: Kaggle experience (Part 1 of 2) </a> de Vlad Shmyhlo en Towards Data Science 
        </td>
    </tr>
</table>

El mar domina quasi tota la imatge, pel que més del 90% de la imatge pertany a la classe de fons, al mar.  Només que detectem correctament aquesta classe ja estarem tenint un resultat molt elevat del Pixel d'Accuracy, tant si detectem algun vaixell com si no.

Aquest problema s'anomena "desequilibri de classe" (en anglès *class imbalance*) i és molt comú en moltes imatges del món real com en la conducció automàtica. Quan les classes a classificar estan molt desequilibrades, la mètrica Pixel Accuracy no ens pot donar valors fiables.


### IoU o Jacquard Index

La "Intersecció sobre Unió" (**IoU** de l'anglès *Intersection over Union*) també es coneix amb el nom de Jacquard Index i ja la vam introduir quan vam parlar de les mètriques d'avaluació per detecció d'objectes (mAP el capítol 10.3 Detecció d'objectes en una fase).

<table width=75%>
    <tr>
        <td style="border:1px solid black" width=45%><img src="Imatges/iou_equation.png"></td>       
        <td style="border:1px solid black"><img src="Imatges/iou_examples.png"></td>       
    </tr>
    <tr>
        <td style="border:1px solid black" align=center>El càlcul de l'IoU és la divisió entre l'àrea d'intersecció i l'àrea d'unió de les capses base i predites.</td>
        <td style="border:1px solid black" align=center>Exemples de valors de IoU.</td>
    </tr>  
    <tr>
        <td colspan=2 align=center style="border:1px solid black">
            Font: <a href="https://www.pyimagesearch.com/2016/11/07/intersection-over-union-iou-for-object-detection/">PyImageSearch</a>
        </td>
    </tr>
</table>

La IoU és una mesura de precisió basada en la quantitat d'àrea de la de la finestra predicció que coincideix amb la finestra d'entrenament (o el "ground truth"). I és una de les mètriques més usades per a la segmentació d'imatges, calculant la mitjana del seu valor per a cada instància o classe a segmentar en la imatge. El seu valor està sempre entre 0 i 1 (essent 1 el màxim).



# Algoritme U-NET

Un dels algoritmes més coneguts per a la segmentació d'imatges és el que es coneix amb el nom de **U-Net** i va ser presentat per Olaf Ronneberger, Phillip Fischer, i Thomas Brox al 2015 com a proposta per a una millor segmentació d'imatges biomèdiques.

Tot i que la U-Net va ser originalment creada per a la segmentació d'imatges mèdiques, funciona molt bé en una gran varietat de problemes de segmentació d'imatges naturals, des de segmentació de cèl·lules en imatges de microscopi fins a la detecció de vaixells i cases en fotos fetes per satèl·lit.

La següent imatge mostra l'arquitectura de la U-Net:

<table width=75%>
    <tr>
        <td>
            <img src="Imatges/unet.png">
        </td>
    </tr>
    <tr>
        <td align=center>
            <i>Font: <a href="https://arxiv.org/pdf/1505.04597.pdf">Olaf Ronneberger et al. "U-Net: Convolutional Networks for Biomedical Image Segmentation" (2015)</a></i>
        </td>
    </tr>
</table>

El nom de l'algoritme ve de la forma de "U" que pren la seva arquitectura. L'algoritme aplica capes, per ordre, des de la part superior esquerra, baixant fins la part inferior, per pujar després per la part dreta.

La proposta consta de dues parts molt diferenciades en l'arquitectura: la codificació o compressió de dades (part esquerra que baixa) i la descodificació o descompressió de dades (part dreta que puja). 

La primera part, la codificació, s'utilitza per captar el context de la imatge i consta d'un conjunt de capes convolucionals i max pooling. La segona part, la descodificació, s'utilitza per localitzar amb precisió fent servir un conjunt de convolucions transposades. 

Arquitectura:
- Codificació: Consta d'una arquitectura típica CNN amb capes consecutives de convolucions 3x3 (fletxa blava) seguida d'un max pooling 2x2 (fletxa vermella). A cada nova capa el nombre de canals es duplica.
- Descodificació: Consta d'una convolució ascendent 2x2 (fletxa verda) i dues convolucions 3x3 (fletxa blava). A cada nova capa el nombre de canals es redueix a la meitat. Com es perden els píxels de les vores en cada convolució, en aquesta fase, després de cada capa, es produeix una concatenació de mapes de característiques amb la capa corresponent del camí de codificació (fletxes verdes), proporcionant aíxi informació de la localització.
- Capa final: Una convolució 1x1 per associar el mapa de característiques amb el nombre desitjat de classes de sortida.

Tot junt crea una FCN (Fully Convolutional Network), és a dir, una xarxa que només conté capes convolucionals, la qual cosa permet, entre d'altres avantatges, acceptar imatges de qualsevol tamany. La xarxa no té capes totalment connectades, només s'utilitzen les capes de convolució, cadascuna d'elles activada mitjançant una funció d'activació de ReLU.

La funció de loss és una binary cross entropy loss ponderant més els píxels que pertanyen a les fronteres dels objectes. Podeu trobar més informació sobre aquesta funció loss en aquesta entrada [Understanding binary cross-entropy / log loss: a visual explanation](https://towardsdatascience.com/understanding-binary-cross-entropy-log-loss-a-visual-explanation-a3ac6025181a) de TDS.

Un dels avantatges de l'U-Net és que no necessita milers d'imatges d'entrenament per poder obtenir uns alts nivells de precisió. Com exemple, us mostrem en la següent imatge els resultats de l'aplicació d'una arquitectura U-Net per detectar glaucomes en imatges de retines; el glaucoma és una de les causes de la ceguera i una de les mesures per detectar-la és el rati entre el disc òptic i la copa òptica. La base de dades de l'exemple contenia només 101 imatges de retina, amb màscares d'anotació del disc òptic i la copa òptica. 50 imatges es van fer servir per entrenar el sistema i 51 per fer les inferències. El model va completar l'entrenament en 11 minuts 33 segons i va aconseguir un IoU de 0.8421.


<table width=75%>
    <tr>
        <td>
            <img src="Imatges/u-net-retines.jpeg">
        </td>
    </tr>
    <tr>
        <td>
            Imatges no utilitzades durant l'entrenament per predir el disc òptic (vermell) i la copa òptica (groc)
            <br>
            Font: article <a href="https://towardsdatascience.com/biomedical-image-segmentation-u-net-a787741837fa">Biomedical Image Segmentation: U-Net</a>, de Hong Jing (Jingles) en Towards Data Science.
        </td>
    </tr>
</table>


# Mask R-CNN
Finalment no podem tancar aquest capítol sense parlar d'un dels algoritmes més populars avui en dia per a la segmentació d'instàncies en imatges: el Mask R-CNN.

En el capítol anterior vam explicar l'evolució rapidíssima que havien tingut els detectors d'objectes de dues fases en poc més de 3 anys: R-CNN (2014), Fast R-CNN (2015) i Faster R-CNN (2016). Doncs bé, l'evolució no es va parar aquí. Al 2017 va aparèixer el **Mask R-CNN** com a extensió del Faster R-CNN. 

El Faster R-CNN consta de dues etapes: una primera etapa anomenada RPN (Region Proposal Network) que proposa candidats a bounding boxes i, una segona etapa, on s'extreuen les característiques de cada candidat a bounding box (amb RoIPool) i on es fa una regressió per calcular la classificació (etiqueta de classe) i les coordenades del bounding box.

El Mask R-CNN afegeix una tercera branca paral·lela en la segona etapa del Faster R-CNN per calcular la màscara de l'objecte detectat. D'aquesta manera s'aconsegueixen 3 sortides per a cada objecte detectat: l'etiqueta de classe, les coordenades de la bounding box i la màscara de l'objecte.

<table width=75%>
    <tr>
        <td>
            <img src="Imatges/mask-r-cnn.png">
        </td>
    </tr>
    <tr>
        <td style="background:white">
            Resultats Mask R-CNN amb el conjunt de test COCO. Es mostren les màscares en color. També es mostren les bounding box i els valors de confiança.
        </td>
    </tr>
    <tr>
        <td align=center>
            <i>Font: <a href="http://openaccess.thecvf.com/content_ICCV_2017/papers/He_Mask_R-CNN_ICCV_2017_paper.pdf">Kaiming He et al. Mask R-CNN (2017)</a></i>
        </td>
    </tr>
</table>


Desavantatge principal: necessita molt temps d'entrenament.


# Referències

- Mètriques d'error en Segmentació d'Imatges: [https://towardsdatascience.com/metrics-to-evaluate-your-semantic-segmentation-model-6bcb99639aa2](https://towardsdatascience.com/metrics-to-evaluate-your-semantic-segmentation-model-6bcb99639aa2)
- Exemple aplicació U-Net per imatges biomèdiques: *Digital Pathology segmentation using Pytorch + U-NET*: [http://www.andrewjanowczyk.com/pytorch-unet-for-digital-pathology-segmentation/](http://www.andrewjanowczyk.com/pytorch-unet-for-digital-pathology-segmentation/)
- Olaf Ronneberger, Philipp Fischer, and Thomas Brox. "U-Net: Convolutional Networks for Biomedical Image Segmentation" (2015): [https://arxiv.org/pdf/1505.04597.pdf](https://arxiv.org/pdf/1505.04597.pdf)
- Kaiming He, Georgia Gkioxari, Piotr Dollar, Ross Girshick. "*Mask R-CNN*" (2017): [http://openaccess.thecvf.com/content_ICCV_2017/papers/He_Mask_R-CNN_ICCV_2017_paper.pdf](http://openaccess.thecvf.com/content_ICCV_2017/papers/He_Mask_R-CNN_ICCV_2017_paper.pdf)
- Exemple d'aplicació Mask R-CNN amb Python: *Computer Vision Tutorial: Implementing Mask R-CNN for Image Segmentation (with Python Code)*, de Pulkit Sharma en Medium: [https://medium.com/analytics-vidhya/computer-vision-tutorial-implementing-mask-r-cnn-for-image-segmentation-with-python-code-fe34da5b99cd](https://medium.com/analytics-vidhya/computer-vision-tutorial-implementing-mask-r-cnn-for-image-segmentation-with-python-code-fe34da5b99cd)
